# Introduction: The Abstract Forest Model

## Introduction 

Prototypical ML tasks: 
* Classification (output is a label)
* Regression (output is a continuous variable)
* Density estimation (estimating a PDF)
* Semi-supervised learning
* Active learning (learning using minimum amount of ground-truths)

Through a unified model of decision forests, we can map the prototypical ML problems onto the same general model by means of different paramterizations.

Decision forests achieve generalization through ensembling slightly different trees.

## Decision Tree Basics

### Tree Data Structure

* A tree is a graph that has a hierachical structure.
* Nodes are either internal (split) or terminal (leaf) nodes.
* Each node (except the root) have exactly one incoming edge.
* A tree doesn't contain loops.

### Decision Tree

* Solving a complex problem by running a series of simpler tests
* For a given input, the decision tree estimates an unknown property of the input by asking successive questions about its known properties
* The next question depend on the answer to the previous question
* The decision is made based on the terminal node
* This mechanism can be graphically represented (path followed by the input through the questions)
* All the questions asked move our decision making towards the correct region of the decision space
* The more question asked, the more higher the confidence in the response
* The tests are represented by a decision tree structure. 
    * Internal nodes: one question/ test each
    * Root node: where the input is injected
    * Leaf node: predictor (contains the most probable answer based on the questions asked during the tree descent)
* Summary: **A decision tree is a tree where each internal node stores a split (or test) function to be applied to the incoming data. Each leaf stores the final answer (predictor). It is a hierachical piecewise model that splits complex problems into simpler ones.** 

### Mathematical Notations and Basic Defitions

#### Data Points and Features

* A data point is given by, 
    $$ \mathbf{v} = (x_1, x_2, \dots, x_d) \in \mathbb{R}^d $$
where $x_i$ is a feature of the data point and $d$ is the feature space dimension (could be infinitely large, but we can pick a subset)
* Features of interest are given by,
    $$ \mathbf{\phi (v)} = (x_{\phi_1}, x_{\phi_2}, \dots, x_{\phi_{d'}}) \in \mathbb{R}^{d'} $$
where $d' << d$ and $\phi_i \in [1, d]$.

#### Test Functions, Split Functions, and Weak Learners

* Each node has a different associated test function.
* Test function (split function, weak learners) at a split node $j$ is given by, 
    $$ h(v, \theta_j): \mathbb{R}^d \times \mathcal{T} \rightarrow \{0, 1\} $$
where, $\theta_j$ are the split parameters of the node $j$.

#### Training Points and Training Sets

* In a training data point, the attributes we are seeking may be known and could be be used to compute tree parameters.
* Definitions: 
    * $\mathcal{S}_0$: Training set
    * $\mathcal{S}_j$: Subset of training points reaching node $j$ (nodes are numbered breadth-first)
    * $\mathcal{S}_j^L$: Subset going to the left child of node $j$
    * $\mathcal{S}_j^R$: Subset going to the left child of node $j$
* Dataset properties: 
    * $\mathcal{S}_j = \mathcal{S}_j^L \cup \mathcal{S}_j^R$
    * $\mathcal{S}_j^L \cap \mathcal{S}_j^R =  \emptyset$
    * $\mathcal{S}_j^L = \mathcal{S}_{2j+1}$
    * $\mathcal{S}_j^R = \mathcal{S}_{2j+2}$

#### Randomly Trained Decision Trees

* **on-line phase: testing**
    * The node tests have been selected (by training) and fixed during testing.
    * Starting at the root, each split node applies the $h(\cdot, \cdot)$ to $\mathbf{v}$. 
    * Depending on the result, $\mathbf{v}$ is sent to the left or right child.
    * This is repeated until a leaf node is reached. 
    * Leaf has a predictor/ estimator (e.g. classifier or a regressor) which assigns a lable to $\mathbf{v}$.
* **off-line phase: training**
    * Split functions could be handcrafted / automatically learnt from data.
    * Training phase select the type and params of the test function $h(v, \theta)$ of each split node $j$ by optimizing a chose objective function on an available training set.
    * This optimization happens in a greedy manner.
    * At each node $j$, based on the incoming training set $\mathcal{S}_j$, a function is learnt that best splits $\mathcal{S}_j$ in to $\mathcal{S}_j^L$ and $\mathcal{S}_j^R$. This problem is written as, 
        $$ \theta_j = \argmax_{\theta \in \mathcal{T}} I(\mathcal{S}_j, \theta) $$
    where, $I$ is an objective function.
    * This is done by a simple search over a discrete set of samples.
    * if $h(v, \theta) = 0$ then $v$ would go to the left child and if not, it would go to the right child.
    * $I$ is computed using the children and parent sets as the inputs. (children are a function of parent and params)
    * For binary classification, best split may occur when the child nodes are as pure as possible (containing training points of a single class). 
    * Making child nodes and partitioning the dataset into disjoint subsets applies recursively to all the newly constructed nodes after the training begins from the root node.
    * The training continues until a stopping criterion is met. The structure of the tree depends on that.
    * Stopping criterion options: 
        * When the three reaches a maximum number of levels 
        * Impose a minimum value of the information gain at the node (when the sought-for-attributes of the training points within the leaf node are similar)
        * When a node contains too few training points
    * Avoiding full grown trees (trees where each leaf containing one training point) helps us generalize the models. 
    * At the end of the training phase, 
        * The optimum weak learners (split functions) associated with each node
        * A learned tree structure 
        * A different set of training points at each leaf

### Weak Learner Models

* Important for both training and testing.
* The params of the weak learner model is 
    $$ \theta = (\phi, \psi, \tau)$$
where $\phi(v)$ selects some features out of $v$, $\psi$ defines the geometric primitives (e.g. a line) used to separate data, and $\tau$ captures the threshold for inequalities in the binary test. 
* These params have to be optimized. 

#### Linear Data Separation

$$ h(v, \theta) = \mathbb{I}\{ \tau_1 > \phi(v) \cdot \psi > \tau_2 \} $$

* Axis-aligned weak learned are often used in the boosting literature
* They are called "decision stumps"
* Axis-aligned cased is overparamterized. 

#### Nonlinear Data Separation

* More complex weak learners are obtained by replacing hyperplanes with hier degree of freedom surfaces.
* E.g. 
    $$ h(v, \theta) = [ \tau_1 > \phi(v)^\top \psi \phi(v) > \tau_2 ]$$
* The number of DOFs of the weak learner influences heavily the forest generalization properties. 

### Energy Models

* Basic building blocks of the training objective function are entropy and information gain. 
* Information gain $I$ at a split node = **reduction in uncertainty** achieved by splitting the data arriving at the node into multiple subsets
    $$ I = H(S) - \sum_{i \in {L, R}} \frac{|S^i|}{|S|} H(S^i) $$
where $H$ is the entropy (a measure of uncertainty associated with the random variable we wish to predict).
* Weighting the entropy using the cardinality of the child sets avoid splitting off childre conraining very few points. 
* For discrete probability distributions, Shannon entropy can be used: 
    $$ H(S) = - \sum_{c \in C} p(c) \log p(C) $$
where $c$ is the class label and $p(c)$ is the empirical distribution extracted from the training points within training data $S$.
* $p(c)$ can be computed as a normalized histogram of the class labels. 
* When the children distributions are more pure, the entropy decreases and the information content increases. 
* Maximizing information gain helps us select the split params whcih produces the highest confidence in final distributions. 
* For continuous probability distributions the differential entropy can be used: 
    $$ H(S) = - \int_{y \in Y} p(y) \log p(y) dy $$
where $y$ is the continuous label and $p(y)$ is the pdf of $y$ estimated from the dataset $S$.
* $p(y)$ can be defined using parametric distibutions or non-parametric methods. 
* Gaussian-based models are frequently used to approximate $p(y)$. 
* The differential entropy of a $d$-variate Gaussian is,
    $$ H(S) = \frac{1}{2} \log (2\pi e)^d | \Lambda (S) | $$
* Overlap between different distributions indicate a suboptimal separation.
* The fact that we can define informaiton gain measures for both continuous and discrete distributions, it is useful to build a unified model.

### Leaf Prediction Models 

* Training phase has to learn good prediction models to be stored at the terminal nodes. 
* In supervising learning, after training, each leaf node has a subset of labels training data. 
* These can be used to compute label statistics in the leaf to predict the label associated with the input test point. 
* Generally, leaf statistics can be capture using conditional distributions $p(c|v)$ or $p(y|v)$.
* Conditioning denotes that the distributions depend on the specific leaf node reached by the test point. 
* MAP estimate can be obtained as, 
    $$ c^* = \argmax_c p(c|v)$$
    $$ y^* = \argmax_y p(y|v)$$
* Don't take ealry point estimates. 

### The Randomness Model

* Randomness is injected to the trees during the training phase
    * Random training set sampling
    * Randomized node optimization

#### Bagging

* Reduces overfitting and improves generailization 
* Trains each tree in a forest on a different training subset sampled at random from the dataset
* Avoid specializing the params to a single training set
* Training is faster
* Not being able to use the entire labeled set is a waste

#### Randomized Node Optimization

* Node optimiaztion w.r.t the entire parameter space $T$ is inefficient when $T$ is large
* Thus, when training node $j$ a small random subset $T_j \subset T$ is selected and optimized
* When $|T| = \infty$, a parameter $\rho = |T_j|$ is introduced to control the degree of randomness in tree (usually the value is fixed for all nodes)
* In practical applications, we may wish to randomize one, some, or all of the parameters $\phi, \psi$ and $\tau$.
* The param values need not be sampled from a uniform distribution.
* Bagging and randomized node optimization can be used together

### Combining Tree into a Forest Ensemble

* A random decision forest is an ensemble of randomly trained decision trees.
* The component trees are ranomly different from each other.
* This lead to decorrelation between individual tree models and improves generlization and robustness.
* The forest model is characterised by,
    * family of weak learners
    * energy model
    * leaf predictors
    * type of randomness influence 
* The randomness parameter $\rho$ controls, 
    * amount of randomness between different trees
    * amount of correlation between different trees in the forest
* In a forest with $T$ trees $t \in \{1, \dots, T \}$ is used to index the trees.
* All the trees are trained indepently and in parallel. 
* Testing can be done in parallel too.
* Combining all the tree predictions into a single forest prediction is done by a simple averaging operation.
    $$ p(c|v) = \frac{1}{T} \sum_{t=1}^{T} p_t(c|v)$$
where $p_t(c|v)$ is the posterior distribution obtained by the $t$ th tree.
* Alternatively, 
    $$ p(c|v) = \frac{1}{Z} \prod_{t=1}^{T} p_t(c|v)$$
where $Z$ is the partition function that ensures the probabilistic normalization.
* Partition function is defined as, 
    $$ Z = \sum_{i} \exp (-\beta E_i) $$
* Peakier distributions are more confident.
* The ensembling output is influenced by the most confident, most informative trees. 
* Averaging many tree posteriors reduce the effect of noisy tree contributions. 
* The product based tree ensemble model produces sharper distributions and may be less robust to noise. 

#### Key Model Parameters

* the maximum allowed tree depth $D$
* the amount of randomness (controlled by $\rho$) and its type 
* the forest size $T$
* the choice of weak learner model
* the training objective function
* the choice of features in practical applications.

#### Other tips

* In general, very unbalanced trees should be avoided. 
* When training a tree, it is important to visualize its trees and other variables (features and params chose at each node)


 













